<h1><b>Introducción a Tensorflow</b> </h1>

<author>Julio Waissman Vilanova</author>

<br/>

<a target="_blank" href="https://colab.research.google.com/github/juliowaissman/intro-rn/blob/main/tf-intro.ipynb">
<img src="https://i.ibb.co/2P3SLwK/colab.png" width=30pt />
<i>Para usar en Google Colab</i></a>

## Introducción

TensorFlow es una biblioteca de software ampliamente utilizada en aprendizaje automático. Aquí aprenderemos cómo se representan los cálculos. TensorFlow 2 ofrece una gran flexibilidad y la capacidad de ejecutar operaciones de manera imperativa. Notarás que TensorFlow 2 es bastante similar a Python en su sintaxis y ejecución imperativa. Instalemos TensorFlow y un par de dependencias.

In [1]:
# Librerías clásicas de python para manejo
# de vectores y matrices, y de graficación
import numpy as np
import matplotlib.pyplot as plt

# Tesorflow
import tensorflow as tf

TensorFlow se llama así porque maneja el flujo (nodo/operación matemática) de tensores, que son estructuras de datos que se pueden considerar como matrices multidimensionales. Los tensores se representan como matrices de n dimensiones de tipos de datos básicos, como una cadena o un número entero; proporcionan una forma de generalizar vectores y matrices a dimensiones superiores.

La *forma* (`shape`) de un Tensor define su número de dimensiones y el tamaño de cada dimensión. El *rango* (`rank`) de un tensor proporciona el número de dimensiones; también puedes considerarlo como el orden o grado del tensor.

Veamos primero los tensores 0-d, de los cuales un escalar es un ejemplo:

In [ ]:
deporte = tf.constant("Tennis", tf.string)
numero = tf.constant(1.41421356237, tf.float64)

print(f"`deporte` es un tensor de rango {tf.rank(deporte).numpy()}")
print(f"`numero` es un tensor de rango {tf.rank(numero).numpy()}")

Y veamos ahora unos de 1 dimensión (vectores):

In [ ]:
sports = tf.constant(["Tennis", "Basketball"], tf.string)
numbers = tf.constant([3.141592, 1.414213, 2.71821], tf.float64)

print(f"`sports` es un tensor de {tf.rank(sports).numpy()} dimensiones con forma: {tf.shape(sports)}")
print(f"`numbers` es un tensor de {tf.rank(numbers).numpy()} dimensiones con forma {tf.shape(numbers)}")

A continuación, consideramos la creación de tensores bidimensionales (es decir, matrices) y de rango superior. Por ejemplo, en visión por computadora se suele utilizar tensores 4-d. Aquí las dimensiones corresponden a la cantidad de imágenes de ejemplo en nuestro lote, la altura de la imagen, el ancho de la imagen y la cantidad de canales de color.

In [10]:
'''TODO: Define un tensor 2-d (Matriz)'''
matrix = # TODO

assert isinstance(matrix, tf.Tensor), "matrix debe ser un objeto tensorflow.Tensor"
assert tf.rank(matrix).numpy() == 2

In [ ]:
'''TODO: Define un tensor 4-d'''
# Usa tf.zeros para inicializar un Tensor de ceros de tamaño 10 x 256 x 256 x 3.
# Puedes pensar que es una pila de 10 imágenes a color (RGB) de 256 x 256 pixeles
images = # TODO

assert isinstance(images, tf.Tensor), "images debe ser un objeto tensorflow.Tensor"
assert tf.rank(images).numpy() == 4, "images debe de ser de rango 4"
assert tf.shape(images).numpy().tolist() == [10, 256, 256, 3], "images tiene la forma incorrecta"

Puedes usar *slicing* para acceder a subtensores de un tensor de rango más alto:

In [ ]:
row_vector = matrix[1]
column_vector = matrix[:,1]
scalar = matrix[0, 1]

print(f"row_vector: {row_vector.numpy()}")
print(f"column_vector: {column_vector.numpy()}")
print(f"scalar: {scalar.numpy()}")

## Calculo de tensores

Una forma conveniente de pensar y visualizar cálculos en TensorFlow es en términos de gráficas. Podemos definir esta gráfica en términos de tensores, que contienen datos, y las operaciones matemáticas que actúan sobre estos tensores en algún orden. Veamos un ejemplo simple y definamos este cálculo usando TensorFlow:

![](https://raw.githubusercontent.com/aamini/introtodeeplearning/master/lab1/img/add-graph.png)

In [ ]:
# Crea los nódo en el grafo e inicializa los valores
a = tf.constant(15)
b = tf.constant(61)

# Suma
c1 = tf.add(a,b)
c2 = a + b # "Sobrecarga" en TensorFlow
print(c1)
print(c2)

Observa cómo hemos creado un gráfica de cálculo que consta de operaciones de TensorFlow ya cómo la salida es un tensor con valor 76; acabamos de crear una gráfica de cálculo que consta de operaciones, las ejecutó y nos devolvió el resultado.

Ahora consideremos un ejemplo un poco más complicado:

![texto alternativo](https://raw.githubusercontent.com/aamini/introtodeeplearning/master/lab1/img/computation-graph.png)

Aquí, tomamos dos entradas, `a`, `b`, y calculamos una salida `e`. Cada nodo en la gráfica representa una operación que toma alguna entrada, realiza algún cálculo y pasa su salida a otro nodo.

Definamos una función simple en TensorFlow para construir esta función de cálculo:

In [ ]:
### Definiendo una gráfica de operaciones en tensorflow ###

def func(a,b):
  '''
  TODO: Define las operaciones para c, d, e
  (usa tf.add, tf.subtract, tf.multiply).
  '''
  c = # TODO
  d = # TODO
  e = # TODO
  return e

Ahora, podemos llamar a esta función para ejecutar el gráfico de cálculo dadas algunas entradas `a`, `b`:

In [ ]:
# Ejemplo de entradas para a y b
a, b = 1.5, 2.5

# Ejecuta el calculo
e_out = func(a,b)

print(e_out)

Observa cómo la salida es un tensor con un valor definido por el cálculo, y que la salida no tiene forma ya que es un valor escalar único.

## Diferenciación automática

La [Diferenciación automática](https://en.wikipedia.org/wiki/Automatic_differentiation) es una de las partes más importantes de TensorFlow y es la columna vertebral del entrenamiento con
[b-prop](https://en.wikipedia.org/wiki/Backpropagation). Usaremos [`tf.GradientTape`](https://www.tensorflow.org/api_docs/python/tf/GradientTape?version=stable) para registrar operaciones y calcular gradientes más adelante.

Cuando se calcular nlos valores de una gráfica de operaciones en TensorFlow, todas las operaciones de paso hacia adelante se graban en una *cinta*. Luego, para calcular el gradiente, la cinta se reproduce al revés. De forma predeterminada, la cinta se descarta después de reproducirla al revés; esto significa que un `tf.GradientTape` particular solo puede calcula un gradiente y las llamadas posteriores arrojan un error de tiempo de ejecución. Sin embargo, podemos calcular múltiples gradientes en el mismo cálculo creando una cinta de gradiente ```persistente```.

Primero, veremos cómo podemos calcular gradientes usando `tf.GradientTape` y acceder a ellos para realizar el cálculo. Definimos la función simple $ z = x^4$ pero usando dos funciones: $z = y^2$ y $y = x^2$ y calculamos el gradiente:

In [19]:
### Cálculo de gradientes con GradientTape ###

# z = y^2
# y = x^2
# dz_dx = 4 * x^3

# Ejemplo: x = 3.0
x = tf.Variable(3.0)

# Inicializa la cinta
with tf.GradientTape() as tape:
  # Define la función
  y = x * x
  z = y * y
# Accesa la cita y encuentra la derivada de y respecto a x
dy_dx = tape.gradient(z, x)

assert dy_dx.numpy() == 4 * x.value()**3

Al entrenar redes neuronales, utilizamos diferenciación y descenso de gradiente estocástico (SGD) para optimizar una función de pérdida.

Ahora que tenemos una idea de cómo se puede usar `GradientTape` para calcular y acceder a derivadas, veremos un ejemplo en el que usamos diferenciación automática y SGD para encontrar el mínimo de $loss=(x-x_f)^2$.

Aquí $x_f$ es una variable para un valor deseado que estamos tratando de optimizar; $L$ representa una pérdida que estamos tratando de minimizar. Si bien podemos resolver claramente este problema analíticamente ($x_{min}=x_f$), considerar cómo podemos calcular esto usando `GradientTape` nos ayuda a entender como funcionan las redes neuronales.

## Taller

**Realiza correctamente el método y grafica en la celda de abajo.** 

In [24]:
### Usando gradiente estocástico para  ###

# Inicializa el valor inicial de x en forma aleatoria
x = tf.Variable([tf.random.normal([1])])

# El valor final esperado
x_f = tf.constant(4.0)

# Parámetros de SGD
EPOCHS = 500 # Número de iteraciones
lr = 1e-2 # Tasa de aprendizaje
history = []

for _ in range(EPOCHS):
  with tf.GradientTape() as tape:
    '''TODO: define la función de pérdida a optimizar'''
    loss = # TODO

  # Obteniendo el gradiente
  grad = tape.gradient(loss, x)

  # Aplicando el decenso de gradiente
  new_x = x - lr * grad

  # Asignando el nuevo valor de x
  x.assign(new_x)

  # Agregando el valor al historial
  history.append(x.numpy()[0])


In [ ]:
# Plot the evolution of x as we optimize towards x_f!
plt.plot(history)
plt.plot([0, 500],[x_f,x_f])
plt.legend(('Predicha', 'Esperada'))
plt.xlabel('Iteración (epoch)')
plt.ylabel('Valor de x')
plt.show()